# Debugging Queries

In a previous notebook, we showed how to write queries against a relational database system.
We mentioned that some operations are pushed down to the source system. But how can we be sure?

This notebook shows how to debug queries in RAW.

This requires us to use the Python API directly.

In [1]:
from rawapi import new_raw_client

cli = new_raw_client()

Let's redo the query that reads data from a PostgreSQL database in the Python API.

We need to use the `query_as` method. The default `query` method returns directly Python objects and as we shall see, we need to obtain a stream.

In [6]:
result = cli.query_as("""
SELECT * FROM READ_PGSQL("psql01", "public", "movies") WHERE startyear=1935 LIMIT 10
""", "csv")

The result object is an input stream of data that can be consumed.

However, it has an extra field, called `query_id`.

In [10]:
result.query_id

'b876bed3-ab38-427e-8632-f1319b13c26f-12'

This is a unique identifier for the query.

It can be used to obtain execution logs of that query.

In [11]:
cli.get_query_logs(result.query_id)

{'id': 'b876bed3-ab38-427e-8632-f1319b13c26f-12',
 'query': '\nSELECT * FROM READ_PGSQL("psql01", "public", "movies") WHERE startyear=1935 LIMIT 10\n',
 'createdAt': [2021, 1, 10, 16, 14, 1, 495000000],
 'state': None,
 'detail': 'output-format: csv',
 'entries': [{'level': 'INFO',
   'message': 'Finished writing output in 460 ms',
   'timestamp': [2021, 1, 10, 16, 14, 2, 343000000]},
  {'level': 'INFO',
   'message': 'Starting to write output...',
   'timestamp': [2021, 1, 10, 16, 14, 1, 880000000]},
  {'level': 'DEBUG',
   'message': 'Finished preparing query execution in 0 ms',
   'timestamp': [2021, 1, 10, 16, 14, 1, 878000000]},
  {'level': 'DEBUG',
   'message': 'Preparing query execution',
   'timestamp': [2021, 1, 10, 16, 14, 1, 876000000]},
  {'level': 'DEBUG',
   'message': 'Finished query compilation in 369 ms',
   'timestamp': [2021, 1, 10, 16, 14, 1, 875000000]},
  {'level': 'DEBUG',
   'message': 'Finished query compilation in 355 ms',
   'timestamp': [2021, 1, 10, 16, 14

Let's search for a specific message:

In [14]:
for entry in cli.get_query_logs(result.query_id)['entries']:
    if 'Query to be pushed down' in entry['message']: print(entry['message'])

Query to be pushed down is:
"""SELECT  "_$248"."tconst" AS "tconst","_$248"."startyear" AS "startyear","_$248"."primarytitle" AS "primarytitle","_$248"."originaltitle" AS "originaltitle","_$248"."primaryname" AS "primaryname" FROM """ + _$247 + """ AS "_$248" WHERE ("_$248"."startyear" = 1935)    LIMIT 10"""


Here we can see the exact query that was sent to the PostgreSQL server, confirming that the filter was indeed pushed down.

The query log is also available in the Web Admin UI:

![Query Log Screenshot](Query%20Log.png)

**Next:** [Python Integration](Python%20Integration.ipynb)